In [1]:
!pip install lxml

In [2]:
import random

# Own libraries
from src.cbr.cbr import CBR
from src.entity.query import Query
from definitions import CASE_LIBRARY as CASE_LIBRARY_PATH
from src.cbr.case_library import CaseLibrary, ConstraintsBuilder

random.seed(10)

ImportError: dlopen(/Users/edisonjairbejaranosepulveda/.local/lib/python3.9/site-packages/lxml/objectify.cpython-39-darwin.so, 0x0002): tried: '/Users/edisonjairbejaranosepulveda/.local/lib/python3.9/site-packages/lxml/objectify.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))

In [11]:
query = Query()
query.set_category("ordinary drink")
query.set_glass("old-fashioned glass")
query.set_ingredients(["orange juice"])
query.set_exc_ingredients(["rum", "grenadine"])
query.set_alc_types(["gin"])
query.set_basic_tastes(["salty"])

# TODO: retrieve 5 recipes similar to the query from the case library
# Random recipes for now
CASE_LIBRARY = CaseLibrary(CASE_LIBRARY_PATH)
CONSTRAINT = ConstraintsBuilder(include_category=query.get_category(), include_glass=query.get_glass())
recipes = random.choices(CASE_LIBRARY.findall(CONSTRAINT), k=5)
print("Done")

Done


In [12]:
query = Query()
query.set_category("ordinary drink")
query.set_glass("old-fashioned glass")
query.set_ingredients(["orange juice"])
query.set_exc_ingredients(["rum", "grenadine"])
query.set_alc_types(["gin"])
query.set_basic_tastes(["salty"])

# TODO: retrieve 5 recipes similar to the query from the case library
# Random recipes for now
CASE_LIBRARY = CaseLibrary(CASE_LIBRARY_PATH)
CONSTRAINT = ConstraintsBuilder(include_category=query.get_category(), include_glass=query.get_glass())
recipes = random.choices(CASE_LIBRARY.findall(CONSTRAINT), k=5)

In [13]:
print(f"Ingredients before: {[e.text for e in recipes[0].ingredients.iterchildren()]}")
print(f"Steps before: {[e.text for e in recipes[0].preparation.iterchildren()]}")

cbr = CBR()
cbr.retrieve(query=query, recipes=recipes)
cbr.adapt()
output = cbr.recipe

print(f"Ingredients after: {[e.text for e in output.ingredients.iterchildren()]}")
print(f"Steps after: {[e.text for e in output.preparation.iterchildren()]}")

Ingredients before: ['soda water', 'prosecco', 'campari']
Steps before: ['build into glass over ice, garnish and serve']
Ingredients after: ['soda water', 'prosecco', 'campari', 'orange juice', 'gin', 'worcestershire sauce']
Steps after: ['build into glass over ice, garnish and serve', 'add 1/2 tsp of ingr5', 'add 1 oz of ingr4', 'add ingr3 to taste']


In [ ]:
# use the class cbr to evaluate the function with the query and the recipes
cbr.evaluate(query, recipes)
# print the result
print(cbr.result)
# print the recipe
print(cbr.recipe)
# print the recipe with the ingredients and steps
print(cbr.recipe.ingredients.text)
print(cbr.recipe.preparation.text)
# print the recipe with the ingredients and steps
print(cbr.recipe.ingredients.text)
# Learn the recipe with the ingredients and steps and the recipe with the ingredients and steps
cbr.learn(query, recipes)
# print the result
print(cbr.result)
# print the score of the recipe and the similarity between the query and the recipe
print(cbr.score)
print(cbr.similarity)